# WIP: Liquidity Measure

## Acceptance Criteria
To be provided later.

## Summary
We need to measure liquidity available on a market in order to see whether market makers are keeping 
their commitment. 
Here we propose a method counts liquidity as the probability weighted average of volume on the book. 
This gives view of liquidity at one instant of time; we then use exponential weighted average over time to obtain the desired measure.

## Terminology
- Instantenaous liquidity $\lambda$, defined below in detail in this spec file.
- `mid price = (best bid - best offer) / 2` (or undefined if either side of the book is empty)
- buy / sell side volume refer to the volume available at a distance from mid price, $V = V(x)$, where $x > 0$ refers to sell side, $x < 0$ refers to buy side and $x$ belongs to the set of all price points available on the book
- probability of volume at distance from mid price trading: $p = p(x)$, this will need to come from risk model (spec update needed)
- auction level buy price $x_{min} < 0$ and auction level sell price $x_{max} > 0$ for will come from auction spec.
- decay parameter $\delta$ which determines how far back in time do we go when averaging instantenaous liquidity.
- weighting parameter $\alpha$ which determines how steep the exponential decay is.

Note that both $\delta$ and $\alpha$  are network wide parameters.

## Details
During auctions (price monitoring auctions, liquidity auctions, frequent batch auctions) we don't have `mid price` as mentioned above. Use the `last trade price` instead. During an opening auction there is no last trade price so we cannot use this measure of liquidity. 

The instantenaous liquidiy should get calculated periodically.
To be more precise this should be after each "event" if a configurable liquidity time step parameter has been exceeded (i.e. the core can recalculate the available liquidity as it sees fit). 

### Understanding probability of trading 
Each risk model gives probability distribution of future price moves, given current price (the risk models are too crude to care whether this is mark or mid-price, so let's say mid). Given mid price $S$ and a time step $\tau > 0$ (expressed as year fraction) the risk model gives, for each "tick" the probability that the price at time $\tau$ from now will be $S_\tau$. Write this is $\mathbb P(S_\tau = x | S) =: f(x)$. The probability that volume at tick $x$ trades is 
$$
p(x) = \sum_{x \leq x' \leq x_{max}} f(x') \quad \text{for $x>S$ and } \quad p(x) = \sum_{x_{min} \leq x' \leq x} f(x') \quad \text{for $x<S$}\,.
$$

### Calculating the instantenaous liquidity

Case 1: no mid price
$\lambda := 0$ if there is no mid price (i.e. when either the buy or sell side of the book are empty)

Case 2: we have mid price
1. Obtain $x_{min}$ and $x_{max}$ from the price monitoring information for the nearest-in-time trigger. 
1. Get the list of possible ticks $x$ s.t. $x_{max} \geq x > S$ values from the order book. Call these $x^+_i$, with $i = 1,\ldots,N^+$. 
1. Get the list of possible ticks $x$ s.t. $x_{min} \leq x < S$ from the order book and call these $x^-_i$, with $i = 1, \ldots , N^-$. 
1. Get the volume $V(x)$ available at each $x = x^-_i$ and $x^+_i$ from the order book.
1. Get the probability $p(x)$ of trading for each of $x = x^-_i$ and $x^+_i$ from the risk model as described above. 


Now you can calculate 
$$
\lambda := 
\min\left(
    \sum_{i=1}^{N^+} V(x^+_i) p(x^+_i), 
    \sum_{i=1}^{N^-} V(x^-_i) p(x^-_i), 
\right)\,.
$$

### Calculating the liquidity provided by a single party

This is the same as above except that $V = V(x)$ isn't the entire volume on the book at that time but rather it's the volume provided by the given participant. The mid-price used is the one for the entire order book, not the one implied by a single party.